# Simulated Annealing Code

In [1]:
from random import random
import numpy as np
from numpy.linalg import norm

INF = 100

In [2]:

def main():

    n = int(input("Enter number of spheres: "))
    max_iterations = int(input("Enter the maximum number of iterations: "))

    print()
    print("SIMULATED ANNEALING")
    print(f"    n: {n}  max_iterations: {max_iterations}\n")

    radius, points = sim_annealing(n, max_iterations)

    print(radius)
    #print(print_matlab_array(points))
    #print(calculate_distances(points))

def initialise_random_points(n, dim=3):
    points = []
    for i in range(n):
        p = []
        for j in range(dim):
            p.append(random())
        points.append(np.array(p))
    return np.array(points)

def calculate_distances(points):
    n = len(points)

    distances = np.zeros((n, n))

    for i in range(n):
        for j in range(i, n):
            d = norm(points[i]-points[j])
            distances[i, j] = d
            distances[j, i] = d

    return distances

def find_smallest(distances, points):
    n = len(distances)

    m = INF
    min_i = None
    min_j = None

    for i in range(n):

        # check distance to all other points
        for j in range(i, n):
            if 0 < distances[i, j] < m:
                m = distances[i, j]
                min_i = i
                min_j = j

        # check distance to edge of the box
        for coord in points[i]:
            if coord < m:
                m = coord
                min_i = i
                min_j = None
            if 1-coord < m:
                m = 1-coord
                min_i = i
                min_j = None

    return m, (min_i, min_j)

def update_distances(distances, points, min_i):
    new_point = points[min_i]
    for k in range(len(points)):
        d = norm(points[k]-new_point)
        distances[k, min_i] = d
        distances[min_i, k] = d
    return distances


In [3]:
# DIFFERENT WAYS OF IMPROVING THE POINT

def rerandomise_point():
    return np.array([random(), random(), random()])

def average_k_neighbours(distances, points, i, k=None, dim=3):
    new_point = np.zeros(dim)
    points_to_use = []

    if k is None:
        k = len(points)

    # get the first k indices (excluding i with distance 0) sorted by distance and append to new list
    for index, distance in sorted(list(enumerate(distances[i, :])), key=lambda x: x[1])[1:k+1]:
        points_to_use.append(points[index])

    for point in points_to_use:
        for axis in range(dim):
            new_point[axis] += point[axis]

    return new_point/k

def jiggle_point(old_point, factor=10, dim=3):
    new_point = np.zeros(dim)
    for coord in range(dim):
        xi = old_point[coord]*(1 + (random()-0.5)/factor)
        if xi <= 0 or xi >= 1:
            return old_point
        new_point[coord] = xi
    return new_point

In [4]:
def sim_annealing(n, max_iterations):
    # initialise n points to random locations
    points = initialise_random_points(n)

    # calculate all distances
    distances = calculate_distances(points)

    # initialise minimum distance
    improvement = INF
    m, min_indexes = find_smallest(distances, points)
    min_i, min_j = min_indexes
    old_m = 100.0
    count = 0

    maximum_minimum = 0.0

    while count<max_iterations:

        print(f"   count: {count:5}    m: {m:5.2}    old_m: {old_m:5.2}    improvement: {m-old_m:5.2}")

        # randomly choose between using the i or j of the minimum distance to move
        if random() > 0.5 or min_j is None:
            min_index = min_i
        else:
            min_index = min_j

        old_point = points[min_index]

        # move the minimum point
        new_point = average_k_neighbours(distances, points, min_index, k=2)             # this average k neighbours thing doesn't take into account the boundaries

        # check whether this is an improvement!
        new_points = points.copy()
        new_points[min_index] = new_point
        new_distances = update_distances(distances, new_points, min_index)
        new_min, new_min_indexes = find_smallest(new_distances, new_points)

        # if it is not an improvement, process over?
        if new_min <= m:
            new_point = jiggle_point(old_point, factor=10*len(str(n)))

        # otherwise, update the info
        points[min_index] = new_point
        distances = update_distances(distances, points, min_index)
        old_m = m
        m, min_indexes = find_smallest(new_distances, new_points)
        min_i, min_j = min_indexes

        if m > maximum_minimum:
            maximum_minimum = m

        # increment the iteration counter
        count += 1

    # final radius will be the smallest distance divided by 2
    radius =  m/2

    print(f"maximum minimum seen: {maximum_minimum:6.2}")
    print(f"final minimum:        {m:6.2}")
    return radius, points

In [6]:
main()

Enter number of spheres: 3
Enter the maximum number of iterations: 100

SIMULATED ANNEALING
    n: 3  max_iterations: 100

   count:     0    m: 0.018    old_m: 1e+02    improvement: -1e+02
   count:     1    m: 0.054    old_m: 0.018    improvement: 0.036
   count:     2    m:  0.13    old_m: 0.054    improvement: 0.073
   count:     3    m:  0.13    old_m:  0.13    improvement: 0.0066
   count:     4    m:  0.13    old_m:  0.13    improvement: -0.0015
   count:     5    m:  0.13    old_m:  0.13    improvement: 0.00036
   count:     6    m:  0.14    old_m:  0.13    improvement: 0.0046
   count:     7    m:  0.14    old_m:  0.14    improvement: 0.0015
   count:     8    m:  0.14    old_m:  0.14    improvement: 0.00041
   count:     9    m:  0.14    old_m:  0.14    improvement: 0.00056
   count:    10    m:  0.14    old_m:  0.14    improvement: 0.00052
   count:    11    m:  0.14    old_m:  0.14    improvement: 1.9e-05
   count:    12    m:  0.14    old_m:  0.14    improvement: 1.9e-05
 